# Collecting Brazil's 2022 presidential election polls data
> A tutorial on how to collect, filter and prepare for analysis Brazil's 2022 presidential election polls data.

- toc: true 
- badges: true
- comments: true
- categories: [data-science]
- hide: false
- search_exclude: false
- image: images/chart-preview.png

## Context

This year [more than 146 million fellow Brazilian](https://www.tse.jus.br/eleitor/estatisticas-de-eleitorado/consulta-quantitativo) are going to choose our new President...

## Data sources

### Pool data
- [Poder360 polls database](https://www.poder360.com.br/banco-de-dados/)
### Voters statistics

- [TSE: Voters statistics by Region](https://www.tse.jus.br/eleitor/estatisticas-de-eleitorado/consulta-quantitativo)
- [TSE: Voters statistics by sex and age](https://www.tse.jus.br/eleitor/estatisticas-de-eleitorado/estatistica-do-eleitorado-por-sexo-e-faixa-etaria)

## EDA

### Importing libraries

In [1]:
import pandas as pd
import scipy.stats as stats
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
data_dir = Path('../assets/data/elections2022')
november_df = pd.read_csv(data_dir / '2021_11.csv', sep=';')
december_df = pd.read_csv(data_dir / '2021_12.csv', sep=';')
december_df.describe()

,pesquisa_id,ano,unidades_federativas_id,cargos_id,tipo_id,turno,instituto_id,cenario_id,candidatos_id,condicao,percentual,margem_mais,margem_menos,num_registro,orgao_registro,qtd_entrevistas,partidos_id,cidade
count,299.000000,299.0,299.0,299.0,299.0,299.000000,299.00000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,0.0,0.0,299.000000,299.000000,0.0
mean,9761.675585,2022.0,6.0,3.0,2.0,1.434783,188.58194,24655.598662,1901.418060,0.341137,18.041137,2.408027,2.408027,NaN,NaN,2111.200669,16.956522,NaN
std,4.821488,0.0,0.0,0.0,0.0,0.496560,130.21952,30.562393,1890.381209,0.474886,17.174248,0.468543,0.468543,NaN,NaN,891.841771,16.514427,NaN
min,9755.000000,2022.0,6.0,3.0,2.0,1.000000,39.00000,24603.000000,209.000000,0.000000,0.000000,2.000000,2.000000,NaN,NaN,1000.000000,1.000000,NaN
25%,9758.000000,2022.0,6.0,3.0,2.0,1.000000,64.00000,24638.500000,706.000000,0.000000,3.000000,2.000000,2.000000,NaN,NaN,1200.000000,1.000000,NaN
50%,9761.000000,2022.0,6.0,3.0,2.0,1.000000,94.00000,24656.000000,973.000000,0.000000,10.000000,2.200000,2.200000,NaN,NaN,2002.000000,15.000000,NaN
75%,9766.000000,2022.0,6.0,3.0,2.0,2.000000,332.00000,24681.000000,1706.500000,1.000000,31.000000,3.000000,3.000000,NaN,NaN,3000.000000,33.000000,NaN
max,9769.000000,2022.0,6.0,3.0,2.0,2.000000,356.00000,24706.000000,6057.000000,1.000000,60.000000,3.200000,3.200000,NaN,NaN,3666.000000,62.000000,NaN


Lets filter our dataframe to have just the columns we will use.

In [3]:
columns_to_use = ["pesquisa_id", "cenario_id","turno", "partido", "candidato", "cenario_descricao", "instituto", "percentual", "data_referencia", "qtd_entrevistas"]
december_simple_df = december_df[columns_to_use].copy()
november_simple_df = november_df[columns_to_use].copy()

##### Getting invalid votes intentions (first turn).

Grouped by `pesquisa_id`, `cenario_id` and `turno == 1`

In [4]:
december_simple_df[(december_simple_df.turno==1) & (december_simple_df.partido.isna())].head(20)

,pesquisa_id,cenario_id,turno,partido,candidato,cenario_descricao,instituto,percentual,data_referencia,qtd_entrevistas
0,9764,24661,1,NaN,branco / nulo / nenhum,cenário 1 - estimulado - 1º turno,Datafolha,8.0,13-16.dez.2021,3666
33,9764,24662,1,NaN,branco / nulo / nenhum,cenário 2 - estimulado - 1º turno,Datafolha,8.0,13-16.dez.2021,3666
34,9764,24662,1,NaN,não sabe,cenário 2 - estimulado - 1º turno,Datafolha,2.0,13-16.dez.2021,3666
41,9764,24661,1,NaN,não sabe,cenário 1 - estimulado - 1º turno,Datafolha,2.0,13-16.dez.2021,3666
48,9760,24647,1,NaN,nenhum / não sabe / não respondeu,cenário 1 - estimulado - 1º turno,Futura,3.7,7-13.dez.2021,2000
50,9760,24647,1,NaN,branco / nulo,cenário 1 - estimulado - 1º turno,Futura,4.8,7-13.dez.2021,2000
62,9760,24648,1,NaN,não sabe / não respondeu,cenário 2 - estimulado - 1º turno,Futura,3.2,7-13.dez.2021,2000
63,9760,24648,1,NaN,branco / nulo,cenário 2 - estimulado - 1º turno,Futura,7.7,7-13.dez.2021,2000
65,9760,24649,1,NaN,nenhum / não sabe / não respondeu,cenário 3 - estimulado - 1º turno,Futura,2.4,7-13.dez.2021,2000
66,9760,24649,1,NaN,branco / nulo,cenário 3 - estimulado - 1º turno,Futura,4.6,7-13.dez.2021,2000


#### Lula vote intentions (first turn)


In [5]:
december_simple_df[(december_simple_df.turno==1) & (december_simple_df.partido=="PT")].head(20)

,pesquisa_id,cenario_id,turno,partido,candidato,cenario_descricao,instituto,percentual,data_referencia,qtd_entrevistas
37,9764,24662,1,PT,Lula,cenário 2 - estimulado - 1º turno,Datafolha,47.0,13-16.dez.2021,3666
46,9764,24661,1,PT,Lula,cenário 1 - estimulado - 1º turno,Datafolha,48.0,13-16.dez.2021,3666
49,9760,24647,1,PT,Lula,cenário 1 - estimulado - 1º turno,Futura,37.9,7-13.dez.2021,2000
59,9760,24648,1,PT,Lula,cenário 2 - estimulado - 1º turno,Futura,39.4,7-13.dez.2021,2000
67,9760,24649,1,PT,Lula,cenário 3 - estimulado - 1º turno,Futura,42.4,7-13.dez.2021,2000
75,9760,24650,1,PT,Lula,cenário 4 - estimulado - 1º turno,Futura,43.8,7-13.dez.2021,2000
106,9758,24637,1,PT,Lula,cenário 2 - estimulado - 1º turno,Ideia Big Data,38.0,6-9.dez.2021,1200
110,9758,24638,1,PT,Lula,cenário 3 - estimulado - 1º turno,Ideia Big Data,38.0,6-9.dez.2021,1200
115,9758,24639,1,PT,Lula,cenário 4 - estimulado - 1º turno,Ideia Big Data,42.0,6-9.dez.2021,1200
128,9758,24636,1,PT,Lula,cenário 1 - estimulado - 1º turno,Ideia Big Data,37.0,6-9.dez.2021,1200


### Add qtdy
$$qtd\_mencoes = (percentual * qtd\_entrevistas)\div 100.00$$

In [6]:
december_simple_df['qtd_mencoes'] = december_simple_df.percentual * december_simple_df.qtd_entrevistas / 100.0
november_simple_df['qtd_mencoes'] = november_simple_df.percentual * november_simple_df.qtd_entrevistas / 100.0


In [7]:
november_simple_df[(november_simple_df.turno==1) & (november_simple_df.partido=="PT") & (november_simple_df.cenario_descricao.str.startswith('cenário 1'))]

,pesquisa_id,cenario_id,turno,partido,candidato,cenario_descricao,instituto,percentual,data_referencia,qtd_entrevistas,qtd_mencoes
3,9750,24567,1,PT,Lula,cenário 1 - estimulado - 1º turno,Vox Populi,44.0,30.out-4.nov.2021,2000,880.000
21,9749,24559,1,PT,Lula,cenário 1 - estimulado - 1º turno,Quaest,48.0,3-6.nov.2021,2063,990.240
42,9751,24572,1,PT,Lula,cenário 1 - estimulado - 1º turno,Paraná Pesquisas,34.9,16-19.nov.2021,2020,704.980
80,9753,24583,1,PT,Lula,cenário 1 - estimulado - 1º turno,Futura,38.6,16-20.nov.2021,2000,772.000
116,9752,24580,1,PT,Lula,cenário 1 - estimulado - 1º turno,PoderData,34.0,22-24.nov.2021,2500,850.000
134,9754,24593,1,PT,Lula,cenário 1 - estimulado - 1º turno,Atlas,42.8,27-29.nov.2021,4401,1883.628


Prior Lula

In [8]:
november_simple_df[(november_simple_df.turno==1) & (november_simple_df.partido=="PT") & (november_simple_df.cenario_descricao.str.startswith('cenário 1'))].sum(axis=0)

pesquisa_id                                                      58509
cenario_id                                                      147454
turno                                                                6
partido                                                   PTPTPTPTPTPT
candidato                                     LulaLulaLulaLulaLulaLula
cenario_descricao    cenário 1 - estimulado - 1º turnocenário 1 - e...
instituto            Vox PopuliQuaestParaná PesquisasFuturaPoderDat...
percentual                                                       242.3
data_referencia      30.out-4.nov.20213-6.nov.202116-19.nov.202116-...
qtd_entrevistas                                                  14984
qtd_mencoes                                                   6080.848
dtype: object

Prior Brancos e Nulos

In [9]:
bn_df = november_simple_df[(november_simple_df.turno==1) & (november_simple_df.partido.isna()) & (november_simple_df.cenario_descricao.str.startswith('cenário 1'))]
bn_df.groupby(['cenario_id']).agg(bn=('qtd_mencoes', 'sum'), total=('qtd_entrevistas', 'max')).sum(axis=0)

bn        1517.332
total    14984.000
dtype: float64

Likelihood Lula

In [10]:
december_simple_df[(december_simple_df.turno==1) & (december_simple_df.partido=="PT") & (december_simple_df.cenario_descricao.str.startswith('cenário 1'))].sum()

pesquisa_id                                                      78097
cenario_id                                                      197252
turno                                                                8
partido                                               PTPTPTPTPTPTPTPT
candidato                             LulaLulaLulaLulaLulaLulaLulaLula
cenario_descricao    cenário 1 - estimulado - 1º turnocenário 1 - e...
instituto            DatafolhaFuturaIdeia Big DataIpecIpespeMDAPode...
percentual                                                       344.7
data_referencia      13-16.dez.20217-13.dez.20216-9.dez.20219-13.de...
qtd_entrevistas                                                  16907
qtd_mencoes                                                   7376.886
dtype: object

Likelihodd brancos e nulos

In [11]:
bn_df = december_simple_df[(december_simple_df.turno==1) & (december_simple_df.partido.isna()) & (december_simple_df.cenario_descricao.str.startswith('cenário 1'))]
bn_df.groupby(['cenario_id']).agg(bn=('qtd_mencoes', 'sum'), total=('qtd_entrevistas', 'max')).sum(axis=0)

bn        1953.256
total    16907.000
dtype: float64

## Playing with the Dirichlet Multinomial

In [17]:
with pm.Model() as dirichlet_model:
    november_prior = pm.Dirichlet(
        "november_prior",
        # Lula, BrancosNulosNaoSabem, Demais candidatos
        # a=pd.array([6080, 1517, 7387]).astype("float32")
        a=pd.array([1.0, 1.0, 1.0]).astype("float32"),
    )
    december_like = pm.Multinomial(
        "december_like", n=16907, p=november_prior, observed=pd.array([7376, 1953, 7578])
    )
    

In [18]:
with dirichlet_model:
   dirichlet_trace = pm.sample(5000) # 100K samples
   ppc = pm.sample_posterior_predictive(
        dirichlet_trace, random_seed=777,
   )

/tmp/ipykernel_1903678/269223508.py:2: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  dirichlet_trace = pm.sample(5000) # 100K samples
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [november_prior]


Sampling 2 chains for 1_000 tune and 5_000 draw iterations (2_000 + 10_000 draws total) took 15 seconds.


In [19]:
print(ppc)
ppc['december_like'].mean(axis=0)

{'december_like': array([[7293., 1951., 7663.],
       [7340., 1949., 7618.],
       [7336., 2015., 7556.],
       ...,
       [7331., 1957., 7619.],
       [7350., 1981., 7576.],
       [7408., 1912., 7587.]])}


array([7375.0836, 1953.2128, 7578.7036])

In [20]:
lula = ppc['december_like'][:, 0]
demais = ppc['december_like'][:, 2]
lula.shape,demais.shape

((10000,), (10000,))

In [21]:
(lula > demais).mean()

0.1207

In [22]:
probs = []
for x in range(30):
    with dirichlet_model:
        dirichlet_trace = pm.sample(5000) # 100K samples
        ppc = pm.sample_posterior_predictive(
                dirichlet_trace, random_seed=777,
        )
    lula = ppc['december_like'][:, 0]
    demais = ppc['december_like'][:, 2]
    probs.append((lula > demais).mean())

/tmp/ipykernel_1903678/1724641766.py:4: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  dirichlet_trace = pm.sample(5000) # 100K samples
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [november_prior]


Sampling 2 chains for 1_000 tune and 5_000 draw iterations (2_000 + 10_000 draws total) took 18 seconds.


/tmp/ipykernel_1903678/1724641766.py:4: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  dirichlet_trace = pm.sample(5000) # 100K samples
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [november_prior]


Sampling 2 chains for 1_000 tune and 5_000 draw iterations (2_000 + 10_000 draws total) took 20 seconds.
The acceptance probability does not match the target. It is 0.8811296021497512, but should be close to 0.8. Try to increase the number of tuning steps.


/tmp/ipykernel_1903678/1724641766.py:4: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  dirichlet_trace = pm.sample(5000) # 100K samples
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [november_prior]


...

Sampling 2 chains for 1_000 tune and 5_000 draw iterations (2_000 + 10_000 draws total) took 9 seconds.
The acceptance probability does not match the target. It is 0.8887629174168021, but should be close to 0.8. Try to increase the number of tuning steps.


/tmp/ipykernel_1903678/1724641766.py:4: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  dirichlet_trace = pm.sample(5000) # 100K samples
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [november_prior]


Sampling 2 chains for 1_000 tune and 5_000 draw iterations (2_000 + 10_000 draws total) took 8 seconds.


/tmp/ipykernel_1903678/1724641766.py:4: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  dirichlet_trace = pm.sample(5000) # 100K samples
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [november_prior]


Sampling 2 chains for 1_000 tune and 5_000 draw iterations (2_000 + 10_000 draws total) took 8 seconds.


In [23]:
pd.np.mean(probs), pd.np.std(probs)

/tmp/ipykernel_1903678/697546233.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  pd.np.mean(probs), pd.np.std(probs)


(0.12026666666666667, 0.0029044027881055963)